In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

def create_knowledge_base(pdf_paths):
    # Load and split documents
    documents = []
    for path in pdf_paths:
        loader = PyPDFLoader(path)
        documents.extend(loader.load())

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    splits = text_splitter.split_documents(documents)

    # Create vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory="./insurance_db"
    )
    return vectorstore

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

class InsuranceChatbot:
    def __init__(self, vectorstore):
        self.llm = ChatOpenAI(model="gpt-4", temperature=0)
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        self.qa_chain = ConversationalRetrievalChain.from_llm(
            self.llm,
            vectorstore.as_retriever(),
            memory=self.memory
        )

    def respond(self, query):
        try:
            result = self.qa_chain({"question": query})
            return result["answer"]
        except Exception as e:
            return ("I'm having trouble answering that. "
                   "Would you like to speak with a human agent?")

In [ ]:
import streamlit as st
from streamlit_chat import message

def main():
    st.title("Insurance Policy Information Chatbot")

    if 'chatbot' not in st.session_state:
        # Initialize with pre-created knowledge base
        vectorstore = Chroma(
            persist_directory="./insurance_db",
            embedding_function=OpenAIEmbeddings()
        )
        st.session_state.chatbot = InsuranceChatbot(vectorstore)

    if 'generated' not in st.session_state:
        st.session_state.generated = []
    if 'past' not in st.session_state:
        st.session_state.past = []

    # Chat input
    user_input = st.text_input("Ask about insurance policies:", key="input")

    if user_input:
        output = st.session_state.chatbot.respond(user_input)
        st.session_state.past.append(user_input)
        st.session_state.generated.append(output)

    # Display conversation
    if st.session_state.generated:
        for i in range(len(st.session_state.generated)):
            message(st.session_state.past[i], is_user=True, key=f"{i}_user")
            message(st.session_state.generated[i], key=f"{i}")

    # Human agent fallback
    if st.session_state.generated and "human agent" in st.session_state.generated[-1].lower():
        st.warning("Please contact our customer service at 1-800-INSURANCE for further assistance.")

if __name__ == "__main__":
    main()